## Cat Facts
This notebook shows how to create, run, and use output from the Cat Facts assistant! This assistant uses a free demonstration web service with no authentication.

The assistant definition enables Code Interpreter which the assistant uses to create a file which is downloaded in the last step.

## First steps
- First, download AntRunLib from Nuget
- At least once, setup the enviroment using **[0-AI-settings](0-AI-settings.ipynb)**

In [1]:
#r "nuget: AntRunnerLib, 0.6.7"

using AntRunnerLib;
using AntRunnerLib.Identity;
using static AntRunnerLib.ClientUtility;
using System.IO;

#!import config/Settings.cs 

var envVariables = Settings.GetEnvironmentVariables();
foreach (var kvp in envVariables)
{
    Environment.SetEnvironmentVariable(kvp.Key, kvp.Value);
}

var config = AzureOpenAIConfigFactory.Get();
var client = GetOpenAIClient(config);


Installed Packages AntRunnerLib, 0.6.7

## List the assistants in your deployment
If you get an error, verify your setup!

In [2]:
var assistants = (await client.AssistantList()).Data;
foreach(var assistant in assistants)
{
    Console.WriteLine(assistant.Name);
}

Cat Facts
McKessonMDTranscripts
Web Search Test
LocalCodeTool
NoteAntsTest
Blob Pirate
Fed Signal Test
OneNote Ants
Web Search Function Demo
TranscriptAnts
McKessonPDFTranscripts
SillyPirate
AssistAnts
Web Search Ants
Python Ants
Outlook Ants


In [3]:
var assistant = (await client.AssistantList())?.Data?.FirstOrDefault(o => o.Name == "Cat Facts");
if(assistant != null) {
    await client.AssistantDelete(assistant.Id);
    Console.WriteLine("Deleted assistant");
}
else
{
    Console.WriteLine("Didn't find cat facts");
}

Deleted assistant


## Create the Cat Facts assistant
The definition of this assistant is located in the ".\AssistantDefinitions\Cat Facts" folder

".\AssistantDefinitions\" is a default path. You can override it by setting the **ASSISTANTS_BASE_FOLDER_PATH** enviroment variable.

"Cat Facts" contains the following files:
```
Cat Facts
│   instructions.md - The assistant's instructions. This file is optional but when present overwrites the value of the instructions property in manifest.json
│   manifest.json - The assistants base settings including model, "model": "gpt-4o". You need to ensure you have deployed whatever models your assistants require
│
└───OpenAPI
        cat-fact.herokuapp.com.json - The swagger definition for the test service 

```        

[See https://github.com/alexwohlbruck/cat-facts](https://github.com/alexwohlbruck/cat-facts)

### Explanation
AssistantUtility.Create acts based on the contents of a folder structure following a convention laid out in these notebooks. In this case, it copied the contents of instructions.md into the assistant manifest.json. It also parsed the cat-facts swagger which follows an Open API schema into tool defintions for the assitant.

In [4]:
var assistantId = await AssistantUtility.Create("Cat Facts", config);
Console.WriteLine(assistantId)

asst_6knnPj2KhnPYURaoc3Zj6ZdE


## Run the Assistant

output.Dialog shows the conversation. This version of AntRunner uses a built-in assistant definition: ConversationUserProxy which acts on behalf of the user to continue the conversation until an answer is produced up to a limit of three thread runs. Upon completion the thread is deleted. 

In [5]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "Cat Facts",
    Instructions = "Tell me something about cats",
};
var output = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
Console.WriteLine(output.Dialog)


User: Tell me something about cats

Assistant: I called the tool named GetCatFacts with these arguments:
```
{}
```
and got this result:
```
[{"status":{"verified":true,"sentCount":1},"_id":"58e008780aac31001185ed05","user":"58e007480aac31001185ecef","text":"Owning a cat can reduce the risk of stroke and heart attack by a third.","__v":0,"source":"user","updatedAt":"2020-08-23T20:20:01.611Z","type":"cat","createdAt":"2018-03-29T20:20:03.844Z","deleted":false,"used":false},{"status":{"verified":true,"sentCount":1},"_id":"58e009390aac31001185ed10","user":"58e007480aac31001185ecef","text":"Most cats are lactose intolerant, and milk can cause painful stomach cramps and diarrhea. It's best to forego the milk and just give your cat the standard: clean, cool drinking water.","__v":0,"source":"user","updatedAt":"2020-08-23T20:20:01.611Z","type":"cat","createdAt":"2018-03-04T21:20:02.979Z","deleted":false,"used":false},{"status":{"verified":true,"sentCount":1},"_id":"588e746706ac2b00110e59ff",

## output.LastMessage
A core perspective of AntRunnerLib is to get the richness of the assistants API with file retrieval, code interpreter, and function calls in the logical form of a chat completion.
```
output.LastMessage // Final answer
```


In [6]:
Console.WriteLine(output.LastMessage)

Here are some interesting facts about cats:

1. Owning a cat can reduce the risk of stroke and heart attack by a third.
2. Most cats are lactose intolerant, and milk can cause painful stomach cramps and diarrhea. It's best to forego the milk and just give your cat clean, cool drinking water.
3. Domestic cats spend about 70 percent of the day sleeping and 15 percent of the day grooming.
4. The frequency of a domestic cat's purr is the same at which muscles and bones repair themselves.
5. Cats are the most popular pet in the United States: There are 88 million pet cats and 74 million dogs.


## Downloading files from code interpreter
In addition to the Cat Facts API, this assistant also uses code interpreter based on its [maifest.json](.\AssistantDefinitions\Cat Facts\manifest.json)

```
{
  "name": "Cat Facts",
  "description": "I look up facts about cats and can also write Python code!",
  "instructions": "",
  "tools": [
    { "type": "code_interpreter" }
  ],
  "tool_resources": {},
  "top_p": 0.95,
  "response_format": "auto",
  "metadata": {
  },
  "model": "gpt-4o",
  "temperature": 0.7
}
```

This prompt instructs the assistant to create a file **"Tell me something about cats and put the results into a file I can download"**. 

As a result the output contains Annotations. You can download the ones with annotation.Type == "file_path" as demonstrated below.

In [7]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "Cat Facts",
    Instructions = "Tell me something about cats and put the results into a file I can download",
    UseConversationEvaluator = false // Turn off the conversation evaluator
};
var output = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
Console.WriteLine(output.Dialog);
Console.WriteLine(output.Annotations[0].FilePathAnnotation.FileId);

// Download files specified in FilePathAnnotation
foreach (var annotation in output.Annotations)
{
    if (annotation.Type == "file_path")
    {
        var fileId = annotation.FilePathAnnotation.FileId;
        var fileName = Path.GetFileName(annotation.Text.Replace("sandbox:/mnt/data/", ""));

        // Retrieve the file content
        var fileContentResponse = await CodeInterpreterFiles.RetrieveFileContent(fileId, config);

        // Check if there was an error during retrieval
        if (fileContentResponse.Error != null)
        {
            throw new Exception($"Error retrieving file content: {fileContentResponse.Error.Message}");
        }

        // Save the file content to the specified download folder
        var filePath = Path.Combine(@".\", fileName);
        File.WriteAllBytes(filePath, fileContentResponse.Content);
        Console.WriteLine($"{filePath}");
    }
}



User: Tell me something about cats and put the results into a file I can download

Assistant: I called the tool named GetCatFacts with these arguments:
```
{}
```
and got this result:
```
[{"status":{"verified":true,"sentCount":1},"_id":"58e008780aac31001185ed05","user":"58e007480aac31001185ecef","text":"Owning a cat can reduce the risk of stroke and heart attack by a third.","__v":0,"source":"user","updatedAt":"2020-08-23T20:20:01.611Z","type":"cat","createdAt":"2018-03-29T20:20:03.844Z","deleted":false,"used":false},{"status":{"verified":true,"sentCount":1},"_id":"58e009390aac31001185ed10","user":"58e007480aac31001185ecef","text":"Most cats are lactose intolerant, and milk can cause painful stomach cramps and diarrhea. It's best to forego the milk and just give your cat the standard: clean, cool drinking water.","__v":0,"source":"user","updatedAt":"2020-08-23T20:20:01.611Z","type":"cat","createdAt":"2018-03-04T21:20:02.979Z","deleted":false,"used":false},{"status":{"verified":true,"

## Clean up

In [8]:
var assistant = (await client.AssistantList())?.Data?.FirstOrDefault(o => o.Name == "Cat Facts");
if(assistant != null) {
    await client.AssistantDelete(assistant.Id);
    Console.WriteLine("Deleted assistant");
}
else
{
    Console.WriteLine("Didn't find MsGraphUserProfile");
}

Deleted assistant
